# Fitting with Iminuit Vs. Emcee

This notebook provides a simple comparison of the `fit_lc` and `mcmc_lc` minimization routines available in `sncosmo` (based on `iminuit` and `emcee` respectively). Comparisons are performed using the SN91bg Model.

In [ ]:
import sys

import sncosmo
from matplotlib import pyplot as plt
from sndata.sdss import sako18
from sndata.des import sn3yr

sys.path.insert(0, '../')
from phot_class import fit_func_wraps, models, utils

sako18.download_module_data()
sn3yr.download_module_data()

sako18.register_filters(True)
sn3yr.register_filters(True)


We build some simple functions to fit data and plot results. Note that we used the wrapped versions of the minimization routines from our analysis pipeline. 

In [ ]:
def iminuit_fit(data, model, vparams, **kwargs):
    result, fitted_model = fit_func_wraps.simple_fit(data, model, vparams, **kwargs)

    xs, d = utils.calc_model_chisq(data, result, fitted_model)
    print(f'chisq / ndof = {xs} / {d} = {xs / d}')
    print(f'ncall = {result.ncall}')
    print(f'message = {result.message}', flush=True)

    fig = sncosmo.plot_lc(data, model=fitted_model, errors=result.errors)
    plt.show()
    
    return result, fitted_model

def emcee_fit(data, model, vparams, **kwargs):    
    result, fitted_model = fit_func_wraps.mcmc_fit(data, model, vparams, **kwargs)

    xs, d = utils.calc_model_chisq(data, result, fitted_model)
    print(f'chisq / ndof = {xs} / {d} = {xs / d}')
    print(f'samples = {len(result.samples)}')

    fig = sncosmo.plot_lc(data, model=fitted_model, errors=result.errors)
    plt.show()
    return result, fitted_model

    

## Demo Data

For simplicity, we start with the example data from sncosmo.

In [ ]:
data = sncosmo.load_example_data()

models.register_sources(force=True)
model = sncosmo.Model(sncosmo.get_source('sn91bg', version='hsiao_phase'))
model.set(z=data.meta['z'], t0=data.meta['t0'])

vparams = ['amplitude', 'x1', 'c']  # parameters of model to vary
bounds={'x1': (0.65, 1.25), 'c': (0, 1)}  # bounds on parameters


We run a fit using the `iminuit` minimization. To avoid some known bugs in `sncosmo` we use the wrapped version of `fit_lc` available in our analysis pipeline. 

In [ ]:
iminuit_result, iminuit_fitted_model = iminuit_fit(data, model, vparams, bounds=bounds)


Next, we fit the same data using an MCMC minimization. Note that we retrieve a much more reasonable value for the `c` parameter while maintaining a similar quality of fit (chi-squared). However, the error value quickly minimizes this excitement.

In [ ]:
mcmc_result, mcmc_fitted_model = emcee_fit(data, model, vparams, bounds=bounds)

## SDSS and DES Data

We repeat the above process, but look at a few light-curves from SDSS or DES.

In [ ]:
use_data = 'sdss'

if use_data == 'sdss':
    filter_func = utils.classification_filter_factory(
            ['SNIa']
        )

    data_iter = sako18.iter_data(filter_func=filter_func)

elif use_data == 'des':
    data_iter = des.iter_data()
    

In [ ]:
data = next(data_iter)
model.set(z=data.meta['z'], t0=0)
mcmc_result, mcmc_fit = iminuit_fit(data, model, model.param_names[1:], bounds=bounds)
mcmc_result, mcmc_fit = emcee_fit(data, model, model.param_names[1:], bounds=bounds)
